In [1]:
import tqdm
import torch
import wandb
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AdamW

C:\Users\Sid\AppData\Local\Temp\ipykernel_41752\451788269.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
wandb.init(project="MobileBERT Model FineTuning Yo!")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: siddiqmoideen07. Use `wandb login --relogin` to force relogin


In [3]:
dataStuff = pd.read_csv("../augmentedDataStuff.csv")
dataStuff

,OCR Contents,Category
0,Invoice Number: INV0786Hotel Name: Spice Garde...,Food Category - Restaurant Bill/Receipt
1,Invoice No- DW/1112/851166\n\nInvoice Date : 2...,Travel Category - Flight Ticket
2,Hotel: Accor\r\nCity: Delhi\r\nNumber of Days:...,Miscellaneous Category - Miscellaneous Bills
3,FOOD INN FINE DINE\nProprietor\nPER =NNIA REST...,Food Category - Restaurant Bill/Receipt
4,= Confirmed Booking Id: NF20115276689422\n\nSU...,Travel Category - Flight Ticket
...,...,...
5161,Indiandit\n\nSERIAL SNA AUTONOBI\n\nr .A.B.Roa...,Fuel Category - Fuel Bill/Receipt
5162,Base Fare\nDistance\nTime\n\nSubtotal\n\nCHARG...,Travel Category - Cab Bill/Receipt
5163,"Dear sanjay kumar,\n\nCongratdatons! Thark you...",Travel Category - Train Ticket
5164,Invoice Number: INV0780Hotel Name: Taste of It...,Food Category - Restaurant Bill/Receipt


In [4]:
clsStuff = list(set(dataStuff["Category"]))
clsStuff

['Travel Category - Bus Ticket',
 'Room Stay Category - Stay Bill/Receipt',
 'Food Category - Restaurant Bill/Receipt',
 'Electronics Category - Electronic Gadgets Bill/Receipt',
 'Travel Category - Flight Ticket',
 'Travel Category - Cab Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Fuel Category - Fuel Bill/Receipt',
 'Travel Category - Train Ticket']

In [5]:
labelEncoder = LabelEncoder()
labelEncoder.fit(clsStuff)
clsDict = dict(zip(clsStuff, labelEncoder.transform(clsStuff)))
clsDict

{'Travel Category - Bus Ticket': 5,
 'Room Stay Category - Stay Bill/Receipt': 4,
 'Food Category - Restaurant Bill/Receipt': 1,
 'Electronics Category - Electronic Gadgets Bill/Receipt': 0,
 'Travel Category - Flight Ticket': 7,
 'Travel Category - Cab Bill/Receipt': 6,
 'Miscellaneous Category - Miscellaneous Bills': 3,
 'Fuel Category - Fuel Bill/Receipt': 2,
 'Travel Category - Train Ticket': 8}

In [6]:
numLabels = len(clsStuff)
batchSize = 16
learningRate = 2e-5
epochs = 250

In [7]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [8]:
model = AutoModelForSequenceClassification.from_pretrained("google/mobilebert-uncased", num_labels=numLabels).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/mobilebert-uncased")

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.dense.weight']
- This IS expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some 

In [9]:
wandb.watch(model)
wandb.config["learning_rate"] = learningRate
wandb.config["batch_size"] = batchSize

In [10]:
trainData, valData = train_test_split(dataStuff, test_size=0.2, random_state=42)
trainData['OCR Contents']

4711    Hotel: Hilton Worldwide\r\nCity: Amritsar\r\nN...
4184    goibibo sao\n\nFlight Ticket - Kolkata to Agar...
1670    Category: Utilities\r\nItem: Internet Bill\r\n...
99      Invoice Number: INV0810Hotel Name: The Hungry ...
3592    Invoice Number: INV0953Hotel Name: Gourmet Pal...
                              ...                        
4426    Invoice Number: INV0558Hotel Name: The Hungry ...
466     ticket ric 0: r07s220168 Gcleartrip\n\nBangalo...
3092    Category: Shopping\r\nItem: Accessories\r\nAmo...
3772    " ‘\n\nSREP iy\n\nCHOKHI DHANI PALACE HOTEL\n\...
860     Category: Healthcare\r\nItem: Medicines\r\nAmo...
Name: OCR Contents, Length: 4132, dtype: object

In [11]:
def prepData(data):
  train_inputs, train_masks, train_labels = [], [], []
  for item, label in zip(data["OCR Contents"], data["Category"]):
    text_content = item
    encoded_data = tokenizer(text_content, padding="max_length", truncation=True, max_length=512)
    train_inputs.append(encoded_data["input_ids"])
    train_masks.append(encoded_data["attention_mask"])
    train_labels.append(label)
  return train_inputs, train_masks, train_labels

In [12]:
def labels2Encoding(labelStuff):
    encodedLabels = labelEncoder.transform(labelStuff)
    return encodedLabels

In [13]:
trainInputs, trainMasks, trainLabels = prepData(trainData)
valInputs, valMasks, valLabels = prepData(valData)

In [14]:
trainLabels

['Miscellaneous Category - Miscellaneous Bills',
 'Travel Category - Bus Ticket',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Food Category - Restaurant Bill/Receipt',
 'Travel Category - Train Ticket',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Room Stay Category - Stay Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Miscellaneous Category - Miscellaneous Bills',
 'Miscellaneous Category - Miscellaneous Bills',
 'Food Category - Restaurant Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Travel Category - Bus Ticket',
 'Miscellaneous Category - Miscellaneous Bills',
 'Travel Category - Flight Ticket',
 'Food Category - Restaurant Bill/Receipt',
 'Food Category - Restaurant Bill/Receipt',
 'Travel Category - Train Ticket',
 'Travel Category - Bus Ticket',
 'T

In [15]:
trainLabels = labels2Encoding(trainLabels)
valLabels = labels2Encoding(valLabels)
trainLabels

array([3, 5, 3, ..., 3, 1, 3])

In [16]:
trainInputs = torch.tensor(trainInputs, dtype=torch.uint8).to(device)
trainMasks = torch.tensor(trainMasks, dtype=torch.uint8).to(device)
trainLabels = torch.tensor(trainLabels, dtype=torch.uint8).to(device)
valInputs = torch.tensor(valInputs, dtype=torch.uint8).to(device)
valMasks = torch.tensor(valMasks, dtype=torch.uint8).to(device)
valLabels = torch.tensor(valLabels, dtype=torch.uint8).to(device)

In [17]:
print(trainMasks.max(), trainInputs.min())

tensor(1, device='cuda:0', dtype=torch.uint8) tensor(0, device='cuda:0', dtype=torch.uint8)


In [18]:
trainDataset = TensorDataset(trainInputs, trainMasks, trainLabels)
trainDataloader = DataLoader(trainDataset, batch_size=batchSize, shuffle=True)

In [19]:
valDataset = TensorDataset(valInputs, valMasks, valLabels)
valDataloader = DataLoader(valDataset, batch_size=batchSize)

In [20]:
optimizer = AdamW(model.parameters(), lr=learningRate)
criterion = torch.nn.CrossEntropyLoss()

c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
def train(model, optimizer, criterion, trainDataloader):
  model.train()
  total_loss = 0
  correct = 0
  for batch_idx, (input_ids, attention_mask, labels) in enumerate(trainDataloader):
    optimizer.zero_grad()
    input_ids = input_ids.long().to(device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    loss = criterion(outputs.logits, labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    predictions = torch.argmax(outputs.logits, dim=1)
    correct += (predictions == labels).sum().item()
  return total_loss / len(trainDataloader), correct / len(trainDataset)


In [22]:
def validate(model, criterion, valDataloader):
  model.eval()
  total_loss = 0
  correct = 0
  all_preds, all_labels = [], []
  with torch.no_grad():
    for batch_idx, (input_ids, attention_mask, labels) in enumerate(valDataloader):
      input_ids = input_ids.long().to(device)
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      loss = criterion(outputs.logits, labels)
      total_loss += loss.item()
      predictions = torch.argmax(outputs.logits, dim=1)
      correct += (predictions == labels).sum().item()
      all_preds.extend(predictions.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())
  accuracy = accuracy_score(all_labels, all_preds)
  precision = precision_score(all_labels, all_preds, average="weighted")
  recall = recall_score(all_labels, all_preds, average="weighted")
  return total_loss / len(valDataloader), accuracy, precision, recall

In [23]:
for epoch in tqdm.trange(epochs, colour = "red", desc = "Epoch(s)"):
  train_loss, train_acc = train(model, optimizer, criterion, trainDataloader)
  val_loss, val_acc, val_precision, val_recall = validate(model, criterion, valDataloader)
  dataDict = {
    "Epoch": epoch+1,
    "Train Loss": train_loss,
    "Train Accuracy": train_acc,
    "Val Loss": val_loss,
    "Val Accuracy": val_acc,
    "Val Precision": val_precision,
    "Val Recall": val_recall
    }
  print(dataDict)
  wandb.log(dataDict)
  model.save_pretrained("tunedModels/trainedMobileBERT - Epoch {}".format(epoch))

Epoch(s):   0%|          | 0/250 [00:00<?, ?it/s]c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Epoch(s):   0%|          | 1/250 [06:43<27:54:42, 403.54s/it]

{'Epoch': 1, 'Train Loss': 58808.53011300757, 'Train Accuracy': 0.031945788964181994, 'Val Loss': 2.83656252347506, 'Val Accuracy': 0.025145067698259187, 'Val Precision': 0.001095746421425259, 'Val Recall': 0.025145067698259187}


c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Epoch(s):   1%|          | 2/250 [13:26<27:47:43, 403.48s/it]

{'Epoch': 2, 'Train Loss': 3.85908310109584, 'Train Accuracy': 0.22241045498547918, 'Val Loss': 1.9319407609792856, 'Val Accuracy': 0.5087040618955513, 'Val Precision': 0.3719182499027138, 'Val Recall': 0.5087040618955513}


c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
Epoch(s):   1%|          | 3/250 [20:06<27:32:51, 401.50s/it]

...

{'Epoch': 248, 'Train Loss': 0.018298539697210323, 'Train Accuracy': 0.9937076476282671, 'Val Loss': 0.6309157344316303, 'Val Accuracy': 0.9129593810444874, 'Val Precision': 0.9137478937019556, 'Val Recall': 0.9129593810444874}


Epoch(s): 100%|█████████▉| 249/250 [26:58:24<06:32, 392.65s/it]

{'Epoch': 249, 'Train Loss': 0.027299026690572253, 'Train Accuracy': 0.9886253630203291, 'Val Loss': 0.5058187169376849, 'Val Accuracy': 0.9061895551257253, 'Val Precision': 0.9086248233204605, 'Val Recall': 0.9061895551257253}


Epoch(s): 100%|██████████| 250/250 [27:04:55<00:00, 389.98s/it]

{'Epoch': 250, 'Train Loss': 0.01431458005974548, 'Train Accuracy': 0.9956437560503388, 'Val Loss': 0.6065659302950933, 'Val Accuracy': 0.9119922630560928, 'Val Precision': 0.9108098753767224, 'Val Recall': 0.9119922630560928}


In [24]:
model.save_pretrained("trainedMobileBERT")